# Importamos las librerías necesarias

In [2]:
import pandas as pd
import psycopg2
import os

## Hacemos la carga de archivos y hacemos las transformaciones necesarias

In [3]:
# 📌 1️⃣ Cargar los archivos CSV y normalizar los datos
files = {
    "2015": "../inputs/data-2015.csv",
    "2016": "../inputs/data-2016.csv",
    "2017": "../inputs/data-2017.csv",
    "2018": "../inputs/data-2018.csv",
    "2019": "../inputs/data-2019.csv",
    "migration": "../inputs/net_migration_by_country.csv"
}

# Lista para almacenar los dataframes de felicidad
happiness_data = []

# Procesar cada archivo de felicidad (2015-2019)
for year, path in files.items():
    if year != "migration":  # Excluir dataset de migración
        df = pd.read_csv(path)
        
        # Renombrar columnas para estandarizarlas
        rename_dict = {
            f"Score {year}": "happiness_score",
            f"GDP {year}": "gdp_per_capita",
            f"Family {year}": "social_support",
            f"Life Expectancy {year}": "healthy_life_expectancy",
            f"Freedom {year}": "freedom_to_make_choices",
            f"Trust {year}": "corruption_perception",
            f"Generosity {year}": "generosity"
        }
        
        # Aplicar cambios de nombre y seleccionar solo las columnas necesarias
        df = df.rename(columns=rename_dict)
        df = df[["Country", "happiness_score", "gdp_per_capita", "social_support",
                 "healthy_life_expectancy", "freedom_to_make_choices", "corruption_perception", "generosity"]]
        
        # Agregar la columna del año
        df["year"] = int(year)
        
        # Agregar a la lista de dataframes
        happiness_data.append(df)

# Unir todos los datasets de felicidad en uno solo
happiness_df = pd.concat(happiness_data, ignore_index=True)

# Renombrar columna "Country" a "country" para que coincida con el dataset de migración
happiness_df = happiness_df.rename(columns={"Country": "country"})

# 📌 2️⃣ Cargar y filtrar los datos de migración
migration_df = pd.read_csv(files["migration"])
migration_df = migration_df[(migration_df["year"] >= 2015) & (migration_df["year"] <= 2019)]

# Renombrar columnas para que coincidan con happiness_df
migration_df = migration_df.rename(columns={
    "country_name": "country",
    "value": "net_migration"
})

# Seleccionar solo las columnas necesarias
migration_df = migration_df[["country", "year", "net_migration"]]

## Conexión a postgre, creación de BBDD y carga de tablas 

Recordar cambiar las credenciales necesarias para hacer la conexión con PostgreSQL y tener este script en la misma carpeta que los dataset (.csv).

In [4]:
# 📌 3️⃣ Conectar a PostgreSQL y crear la base de datos "avance_2" si no existe
def crear_base_de_datos():
    try:
        conn = psycopg2.connect(
            dbname="postgres",
            user="postgres",
            password="1234",
            host="localhost",
            port="5432"
        )
        conn.autocommit = True
        cursor = conn.cursor()

        # Crear la base de datos si no existe
        cursor.execute("SELECT 1 FROM pg_database WHERE datname = 'avance_2';")
        existe = cursor.fetchone()

        if not existe:
            cursor.execute("CREATE DATABASE avance_2;")
            print("✅ Base de datos 'avance_2' creada correctamente.")
        else:
            print("⚠️ La base de datos 'avance_2' ya existe.")

        cursor.close()
        conn.close()
    except Exception as e:
        print(f"❌ Error al crear la base de datos: {e}")

crear_base_de_datos()

# 📌 4️⃣ Conectar a la nueva base de datos "avance_2"
def conectar_a_base_de_datos():
    try:
        conn = psycopg2.connect(
            dbname="avance_2",
            user="postgres",
            password="1234",
            host="localhost",
            port="5432"
        )
        print("✅ Conexión exitosa a la base de datos 'avance_2'.")
        return conn
    except Exception as e:
        print(f"❌ Error al conectar a la base de datos: {e}")
        return None

conn = conectar_a_base_de_datos()
cursor = conn.cursor()

# 📌 5️⃣ Crear las tablas en PostgreSQL
cursor.execute("""
    CREATE TABLE IF NOT EXISTS happiness_report (
        id SERIAL PRIMARY KEY,
        country TEXT,
        year INT,
        happiness_score FLOAT,
        gdp_per_capita FLOAT,
        social_support FLOAT,
        healthy_life_expectancy FLOAT,
        freedom_to_make_choices FLOAT,
        corruption_perception FLOAT,
        generosity FLOAT
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS migration_data (
        id SERIAL PRIMARY KEY,
        country TEXT,
        year INT,
        net_migration INT
    )
""")
conn.commit()
print("✅ Tablas 'happiness_report' y 'migration_data' creadas correctamente en 'avance_2'.")

# 📌 6️⃣ Insertar datos en la tabla happiness_report
print("⏳ Insertando datos en happiness_report...")
for _, row in happiness_df.iterrows():
    cursor.execute("""
        INSERT INTO happiness_report (country, year, happiness_score, gdp_per_capita, social_support, 
                                      healthy_life_expectancy, freedom_to_make_choices, corruption_perception, 
                                      generosity)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (row["country"], row["year"], row["happiness_score"], row["gdp_per_capita"], 
          row["social_support"], row["healthy_life_expectancy"], row["freedom_to_make_choices"],
          row["corruption_perception"], row["generosity"]))

print("✅ Datos insertados en happiness_report.")

# 📌 7️⃣ Insertar datos en la tabla migration_data
print("⏳ Insertando datos en migration_data...")
for _, row in migration_df.iterrows():
    cursor.execute("""
        INSERT INTO migration_data (country, year, net_migration)
        VALUES (%s, %s, %s)
    """, (row["country"], row["year"], row["net_migration"]))

print("✅ Datos insertados en migration_data.")

# 📌 8️⃣ Confirmar y cerrar conexión
conn.commit()
cursor.close()
conn.close()
print("✅ Conexión cerrada.")

✅ Base de datos 'avance_2' creada correctamente.
✅ Conexión exitosa a la base de datos 'avance_2'.
✅ Tablas 'happiness_report' y 'migration_data' creadas correctamente en 'avance_2'.
⏳ Insertando datos en happiness_report...
✅ Datos insertados en happiness_report.
⏳ Insertando datos en migration_data...
✅ Datos insertados en migration_data.
✅ Conexión cerrada.
